# Genetics

In [7]:
import random
from deap import base
from deap import creator
from deap import tools
import pandas as pd
from sklearn.preprocessing import normalize

In [8]:
pred = pd.read_csv('data/full/predictions.csv')

FileNotFoundError: File b'data/full/predictions.csv' does not exist

In [9]:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

/Users/Lorenzo/anaconda/lib/python3.6/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


In [4]:
def CheckNorm():
    def decorator(func):
        def wrapper(*args, **kargs):
            ind = func(*args, **kargs)
            ind = ind/ind.sum()
            return ind.sum()
        return wrapper
    return decorator

In [5]:
IND_SIZE = 2

toolbox = base.Toolbox()
# Attribute generator 
toolbox.register("attr_float", random.random)
# Structure initializers
toolbox.register("individual", tools.initRepeat, creator.Individual,
                 toolbox.attr_float, n=IND_SIZE)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.decorate("individual", CheckNorm())

In [11]:
def fitness(pred, target):
    
    fit = 0
    if pred.index(max(pred)) == target:
        fit += 1
    fit += pred[target] ** 2
    
    return fit

In [12]:
def compute_candidate(individual, elem):
    
    first = elem[1:6] * individual[:5]
    second = elem[6:11] * individual[5:]
    
    candidate = (first + second)
    candidate = candidate/candidate.sum()
    
    return candidate

In [13]:
def evalOneMax(individual):
    
    fit = 0
    for elem in df:
        candidate = compute_candidate(individual, elem)
        fit += fitness(candidate, elem['target'])
    
    return fit.mean()

In [ ]:
toolbox.register("evaluate", evalOneMax)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

In [ ]:
pop = toolbox.population(n=300)

In [ ]:
# Evaluate the entire population
fitnesses = list(map(toolbox.evaluate, pop))
for ind, fit in zip(pop, fitnesses):
    ind.fitness.values = fit

In [ ]:
CXPB, MUTPB = 0.8, 0.1

In [ ]:
# Extracting all the fitnesses of 
fits = [ind.fitness.values[0] for ind in pop]

In [ ]:
# Variable keeping track of the number of generations
g = 0
    
# Begin the evolution
while max(fits) < 100 and g < 1000:
    # A new generation
    g = g + 1
    print("-- Generation %i --" % g)
    # Select the next generation individuals
    offspring = toolbox.select(pop, len(pop))
    # Clone the selected individuals
    offspring = list(map(toolbox.clone, offspring))
    # Apply crossover and mutation on the offspring
    for child1, child2 in zip(offspring[::2], offspring[1::2]):
        if random.random() < CXPB:
            toolbox.mate(child1, child2)
            del child1.fitness.values
            del child2.fitness.values

    for mutant in offspring:
        if random.random() < MUTPB:
            toolbox.mutate(mutant)
            del mutant.fitness.values
    # Evaluate the individuals with an invalid fitness
    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit
        
    pop[:] = offspring
    
    # Gather all the fitnesses in one list and print the stats
    fits = [ind.fitness.values[0] for ind in pop]

    length = len(pop)
    mean = sum(fits) / length
    sum2 = sum(x*x for x in fits)
    std = abs(sum2 / length - mean**2)**0.5

    print("  Min %s" % min(fits))
    print("  Max %s" % max(fits))
    print("  Avg %s" % mean)
    print("  Std %s" % std)